# Homework 2

## Interior point method implementation

In [16]:
import numpy as np

### 3)

In [17]:
# Compute 1 step of the iteration
def interior_point_method_step(A, b, c, x, y, s, mew):

    m = A.shape[1]
    # the mew update 
    mew = (1 - 1/(6 * np.sqrt(m))) * mew

    # Prepare the matrices for the Newton method
    S_inv = np.diag(1 / s)
    X = np.diag(x)
    ons = np.ones(m)
     
    # Solve for update 
    Dy = np.linalg.solve(A @ S_inv @ X @ A.T, b - mew * A @ S_inv @ ons)
    Ds = -A.T @ Dy
    Dx = -X @ S_inv @ Ds + mew * S_inv @ ons - x

    # Update
    x += Dx
    y += Dy
    s += Ds

    return x,y,s,mew



### 6) 

In [18]:
A = np.array([
    [3, 3, 3, 0, 0],
    [3, 1, 0, 1, 0],
    [1, 4, 0, 0, 1]
], dtype=float)

b = np.array([4, 3, 4], dtype=float)

c = np.array([-3, -4, 0, 0, 0], dtype=float)
x = np.array([2/5, 8/15, 2/5, 19/15, 22/15], dtype=float)
y = np.array([-4/5, -4/5, -2/3], dtype=float)
s = np.array([37/15, 28/15, 12/5, 4/5, 2/3], dtype=float)
mew = 1

# Checks
print("Ax = b ?", np.allclose(A @ x, b))
print("A^T y + s = c ?", np.allclose(A.T @ y + s, c))
print("x > 0 ?", np.all(x > 0))
print("s > 0 ?", np.all(s > 0))

Ax = b ? True
A^T y + s = c ? True
x > 0 ? True
s > 0 ? True


### 7)

In [19]:
# mew = avg
print(np.sum((x * s/np.mean(x * s) - 1)**2))

# mew = 1
np.sum((x * s/1 - 1)**2)

0.0016232448664881226


np.float64(0.002469135802469144)

### 8)

In [15]:
# Iterating until convergence
epsilon = 1e-8
i = 0

while True:
    x_new, y_new, s, mew = interior_point_method_step(A, b, c, x, y, s, mew)
    
    if np.linalg.norm(x_new - x) < epsilon and np.linalg.norm(y_new - y) < epsilon:
        break

    x = x_new
    y= y_new
    i+=1


print("Optimal x:", x)
print(f"y: {y}")
print(f"s: {s}")
print(f"mew: {mew}")
print(f"it took {i} iterations")

KeyboardInterrupt: 